# **Bottle Cap Detection**

***Model Development, Experimentation, and Evaluation***


This notebook provides a step-by-step explanation of the process of developing an object detection model to detect bottle cap colors: dark blue, light blue, and mixed/others.

The model was trained using YOLOv8 Nano, optimized for edge-device inference (target ≤ 10 ms), using a small dataset that was strengthened through augmentation and intensive preprocessing.

#

## **1.   Introduction**

This project focuses on building an automated system to detect the color of bottle caps using deep learning.  
The primary objectives include:

###  Project Goals
1. Dataset preparation, labeling, preprocessing, and augmentation.
2. Training an optimized YOLOv8 nano model using transfer learning.
3. Performing model evaluation and performance analysis.
4. Running inference tests and optimizing inference speed for edge devices such as Raspberry Pi.

YOLOv8n (Nano) is selected due to its efficiency and fast inference time.

## **2. Dataset and Data Preparation**

**Original Dataset**
The given dataset initially contained 12 images https://drive.google.com/file/d/1vbMFTe2E5OHp2h5o_YL-RUoIFKxsrZWT, and I manually added 3 additional images:
- dark blue bottle cap
- light blue bottle cap
- mixed dark–light blue bottle cap

**Preprocessing Steps**

| Preprocessing | Description |
|--------------|-------------|
| Auto-orient | Ensures correct rotation based on EXIF data |
| Resize | Creates uniform image dimensions |
| Auto-adjust contrast | Enhances visibility under varied lighting |
| Tile 2×2 | Generates additional samples by tiling the image |

**Augmentation Pipeline**

| Augmentation | Purpose |
|--------------|---------|
| Flip | Orientation diversity |
| 90° Rotation | Simulates rotated scenes |
| Saturation | Varies color intensity |
| Brightness | Simulates lighting variations |

**Final Dataset Statistics**

After preprocessing + augmentation:

- Total images: 148  
- Train: 132  
- Validation: 12  
- Test: 4  

This approach is essential due to the small size of the original dataset.


In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.4 MB/s eta 0:00:00


In [5]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="VggbhYZy22TlPnh2s5qA")
project = rf.workspace("afifah-apriliani-yurs5").project("bottle-cap-detection-nndn0")
version = project.version(12)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [6]:
import os
from pathlib import Path
from collections import defaultdict

def count_dataset_files(main_folder_path: str):
    """
    Menghitung jumlah file gambar (.jpg) dan label (.txt)
    di sub-folder 'train', 'valid', dan 'test' dalam dataset.

    Args:
        main_folder_path (str): Path ke folder utama dataset (misalnya, 'Bottle-Cap-Detection-12').
    """
    # Menggunakan Path untuk penanganan path yang lebih baik
    base_path = Path(main_folder_path)

    # Memastikan folder utama ada
    if not base_path.is_dir():
        print(f"ERROR: Folder utama tidak ditemukan di: {main_folder_path}")
        return

    # Struktur data untuk menyimpan hasil perhitungan
    stats = defaultdict(lambda: {"images": 0, "labels": 0})

    # Sub-folder yang akan dianalisis
    data_splits = ["train", "valid", "test"]

    print(f"Menganalisis Struktur Dataset di: {main_folder_path}")
    print("-" * 40)

    for split in data_splits:
        split_path = base_path / split

        # Paths ke folder images dan labels
        images_path = split_path / "images"
        labels_path = split_path / "labels"

        # Periksa apakah folder split ada
        if not split_path.is_dir():
            print(f"Peringatan: Folder '{split}' tidak ditemukan, melewati.")
            continue

        # 1. Hitung file gambar (.jpg)
        if images_path.is_dir():
            image_count = len(list(images_path.glob("*.jpg")))
            # Catatan: Tambahkan "*.png" atau format lain jika diperlukan
            stats[split]["images"] = image_count
        else:
            print(f"Peringatan: Folder 'images' di '{split}' tidak ditemukan.")

        # 2. Hitung file label (.txt)
        if labels_path.is_dir():
            label_count = len(list(labels_path.glob("*.txt")))
            stats[split]["labels"] = label_count
        else:
            print(f"Peringatan: Folder 'labels' di '{split}' tidak ditemukan.")

    ## Hasil Statistik
    print("\nHasil Statistik Dataset Bottle-Cap-Detection-12:")
    print("=" * 60)
    print(f"| {'Dataset Split':<15} | {'Jumlah Gambar (.jpg)':<25} | {'Jumlah Label (.txt)':<15} |")
    print("=" * 60)

    total_images = 0
    total_labels = 0

    for split in data_splits:
        img_count = stats[split]["images"]
        lbl_count = stats[split]["labels"]

        total_images += img_count
        total_labels += lbl_count

        print(f"| {split:<15} | {img_count:<25} | {lbl_count:<15} |")

    print("=" * 60)
    print(f"| {'TOTAL':<15} | {total_images:<25} | {total_labels:<15} |")
    print("=" * 60)

# Tentukan folder utama dataset Anda
MAIN_DATA_FOLDER = "Bottle-Cap-Detection-12"

# Jalankan fungsi
count_dataset_files(MAIN_DATA_FOLDER)

Menganalisis Struktur Dataset di: Bottle-Cap-Detection-12
----------------------------------------

Hasil Statistik Dataset Bottle-Cap-Detection-12:
| Dataset Split   | Jumlah Gambar (.jpg)      | Jumlah Label (.txt) |
| train           | 132                       | 132             |
| valid           | 12                        | 12              |
| test            | 4                         | 4               |
| TOTAL           | 148                       | 148             |


## **3. Model Selection: YOLOv8 Nano**

### Why YOLOv8n?
YOLOv8 Nano is highly efficient and suitable for real-time detection on constrained hardware.

### Advantages
✔ Very fast inference (3–10 ms)  
✔ Small model size (<9 MB)  
✔ Well-suited for transfer learning on small datasets  

### Limitations
✘ Lower accuracy compared to larger YOLO variants  
✘ Sensitive to lighting variations (important for color detection)  


## **4. Training Configuration**

The model is fine-tuned with the following parameters:

- **Epochs:** 250  
- **Image Size:** 416  
- **Batch Size:** 16  
- **Early Stopping Patience:** 50  
- **Learning Rate:** 0.0001 (reduced to prevent overfitting)  

### Additional Augmentations
The following augmentations improve generalization:

- HSV color jitter
- Mosaic (0.5)
- Mixup (0.1)
- Copy-paste augmentation
- Random rotation, shear, and perspective transform

This configuration is optimized for **small-data training** with heavy augmentation.


In [8]:
"""
Bottle Cap Detection Training Script
Optimized for edge device inference (5-10ms target)
Optimized for Small Dataset (100 samples)
"""

import os
import torch
from ultralytics import YOLO
import wandb
from datetime import datetime
import time
import cv2

# ==============================================================================
# 1. Configuration
# ==============================================================================
# Sesuaikan parameter ini
CONFIG = {
    'model': 'yolov8n.pt',  # Ganti ke YOLOv8 Nano: Model yang stabil dan cepat
    'data': 'Bottle-Cap-Detection-12/data.yaml',
    'epochs': 250,          # Mengurangi epoch awal, biarkan Early Stopping yang bekerja
    'imgsz': 320,           # Input size, bisa dicoba 320 untuk kecepatan ekstrem
    'batch': 16,            # Batch size ideal
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'patience': 50,         # Ditingkatkan. Lebih sabar untuk dataset kecil
    'save_period': 50,
    'project': 'runs/train_bottle_cap',
    'name': f'yolov8n_cap_{datetime.now().strftime("%Y%m%d_%H%M%S")}',

    # 💥 Augmentation: Ditingkatkan untuk mengatasi dataset 100 sampel (Sangat Penting!)
    # Nilai di bawah ini adalah default/direkomendasikan untuk dataset kecil
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 15.0,        # Rotasi lebih besar
    'translate': 0.15,
    'scale': 0.7,           # Skala lebih bervariasi
    'shear': 5.0,           # Geseran lebih besar
    'perspective': 0.0005,  # Sedikit perspektif
    'flipud': 0.1,          # Sedikit flip vertikal
    'fliplr': 0.5,
    'mosaic': 0.5,          # Mengurangi Mosaic agar lebih banyak gambar asli terlihat
    'mixup': 0.1,           # Diaktifkan. Penting untuk dataset kecil!
    'copy_paste': 0.1,      # Diaktifkan (membutuhkan segmen mask di data.yaml)

    # Fine-Tuning Optimization (Sangat Penting untuk Transfer Learning)
    'lr0': 0.0001,          # Learning Rate Awal DITURUNKAN drastis untuk fine-tuning
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
}

# ==============================================================================
# 2. Fungsi Utama Pelatihan
# ==============================================================================
def train_model():
    """Train YOLOv8 model for bottle cap detection"""

    print("=" * 50)
    print("🚀 Bottle Cap Detection Training (YOLOv8n)")
    print(f"Dataset Size: ~100. Augmentation is CRUCIAL.")
    print("=" * 50)
    print(f"Device: {CONFIG['device']}")
    print(f"Base Model: {CONFIG['model']} (Pre-trained)")
    print(f"Fine-Tuning LR: {CONFIG['lr0']}")
    print(f"Patience (Early Stop): {CONFIG['patience']}")
    print("=" * 50)

    # Initialize Weights & Biases
    wandb.init(
        project="bottle-cap-detection",
        config=CONFIG,
        name=CONFIG['name']
    )

    # Load model (YOLOv8n pre-trained on COCO)
    # Ini melakukan Transfer Learning
    model = YOLO(CONFIG['model'])

    # Train
    results = model.train(
        data=CONFIG['data'],
        epochs=CONFIG['epochs'],
        imgsz=CONFIG['imgsz'],
        batch=CONFIG['batch'],
        device=CONFIG['device'],
        patience=CONFIG['patience'],
        save_period=CONFIG['save_period'],
        project=CONFIG['project'],
        name=CONFIG['name'],

        # Augmentation (dikutip dari CONFIG)
        **{k: CONFIG[k] for k in CONFIG if k in ['hsv_h', 'hsv_s', 'hsv_v', 'degrees', 'translate', 'scale', 'shear', 'perspective', 'flipud', 'fliplr', 'mosaic', 'mixup', 'copy_paste']},

        # Optimization (dikutip dari CONFIG)
        optimizer='Adam',
        lr0=CONFIG['lr0'],
        lrf=CONFIG['lrf'],
        momentum=CONFIG['momentum'],
        weight_decay=CONFIG['weight_decay'],

        # Other
        plots=True,
        verbose=False, # Set False untuk output yang lebih bersih, True untuk detail
    )

    # Validation setelah pelatihan selesai
    print("\n" + "=" * 50)
    print("📊 Validating final model...")
    print("=" * 50)

    # Log hasil ke W&B di akhir
    final_val_results = model.val()

    # Log metrik final ke W&B
    wandb.log({
        "final_mAP50": final_val_results.box.map50,
        "final_mAP50-95": final_val_results.box.map,
    })

    # Save best model path
    best_model_path = os.path.join(CONFIG['project'], CONFIG['name'], "weights", "best.pt")
    print(f"\n✅ Best model saved: {best_model_path}")

    return model, results, best_model_path

# ==============================================================================
# 3. Fungsi Benchmarking dan Export (Tidak berubah, sudah bagus)
# ==============================================================================

def benchmark_inference(model_path, test_image='tes2.jpg', num_runs=100):
    """
    Benchmark inference speed
    ... (Fungsi ini tetap sama)
    """
    # (Isi fungsi benchmark_inference Anda di sini)

    print("\n" + "=" * 50)
    print("⚡ Benchmarking Inference Speed")
    print("=" * 50)

    model = YOLO(model_path)

    img = cv2.imread(test_image)
    if img is None:
        print(f"⚠️ Test image not found: {test_image}. Skipping benchmark.")
        return 0, 0

    print("Warming up...")
    for _ in range(10):
        _ = model(img, imgsz=CONFIG['imgsz'], verbose=False)

    print(f"Running {num_runs} inferences...")
    times = []

    for _ in range(num_runs):
        start = time.time()
        _ = model(img, imgsz=CONFIG['imgsz'], verbose=False)
        end = time.time()
        times.append((end - start) * 1000)

    avg_time = sum(times) / len(times)
    min_time = min(times)
    max_time = max(times)
    fps = 1000 / avg_time

    print(f"\n📊 Inference Statistics:")
    print(f"Average: {avg_time:.2f} ms")
    print(f"Min: {min_time:.2f} ms")
    print(f"Max: {max_time:.2f} ms")
    print(f"FPS: {fps:.2f}")

    if avg_time <= 10:
        print(f"✅ PASSED: {avg_time:.2f}ms ≤ 10ms target")
    else:
        print(f"⚠️ FAILED: {avg_time:.2f}ms > 10ms target")
        print("💡 Consider:")
        print("   - Export to ONNX/TensorRT")
        print("   - INT8 quantization")
        print("   - Smaller input size (e.g., 320x320, adjust CONFIG['imgsz'])")

    wandb.log({
        "avg_inference_time_ms": avg_time,
        "min_inference_time_ms": min_time,
        "max_inference_time_ms": max_time,
        "fps": fps
    })

    return avg_time, fps


def export_optimized_model(model_path):
    """
    Export model to optimized formats for edge deployment
    ... (Fungsi ini tetap sama)
    """
    # (Isi fungsi export_optimized_model Anda di sini)
    print("\n" + "=" * 50)
    print("📦 Exporting Optimized Models")
    print("=" * 50)

    model = YOLO(model_path)

    # Export to ONNX
    print("\n🔧 Exporting to ONNX...")
    onnx_path = model.export(format='onnx', imgsz=CONFIG['imgsz'], simplify=True)
    print(f"✅ ONNX model saved: {onnx_path}")

    # Export to TorchScript
    print("\n🔧 Exporting to TorchScript...")
    torchscript_path = model.export(format='torchscript', imgsz=CONFIG['imgsz'])
    print(f"✅ TorchScript model saved: {torchscript_path}")

    # Try TensorRT export (if available)
    try:
        print("\n🔧 Exporting to TensorRT...")
        # Gunakan 'half=True' untuk FP16 (lebih cepat)
        trt_path = model.export(format='engine', imgsz=CONFIG['imgsz'], half=True)
        print(f"✅ TensorRT model saved: {trt_path}")
    except Exception as e:
        print(f"⚠️ TensorRT export failed: {e}")
        print("   (This is normal if TensorRT is not installed)")

    return onnx_path


def analyze_model_performance(model_path, data_yaml):
    """
    Analyze model performance per class
    ... (Fungsi ini tetap sama)
    """
    # (Isi fungsi analyze_model_performance Anda di sini)
    print("\n" + "=" * 50)
    print("🔍 Analyzing Per-Class Performance")
    print("=" * 50)

    model = YOLO(model_path)
    # Gunakan split='val' untuk konsistensi
    results = model.val(data=data_yaml, split='val', verbose=True)

    # Ambil nama kelas dari file data.yaml (lebih dinamis)
    class_names = results.names

    print("\nPer-Class Metrics:")
    print("-" * 50)

    # Cek apakah metrik per kelas tersedia
    if results.box.ap.shape[0] > 0:
        for i, name in class_names.items():
            # Pastikan indeks ada
            if i < len(results.box.ap):
                print(f"Class {i} ({name}):")
                print(f"  Precision: {results.box.p[i]:.4f}")
                print(f"  Recall: {results.box.r[i]:.4f}")
                print(f"  mAP@50: {results.box.ap50[i]:.4f}")
                print(f"  mAP@50-95: {results.box.ap[i]:.4f}")
    else:
        print("⚠️ Per-Class metrics not available. Check your validation set.")

    return results

# ==============================================================================
# 4. Eksekusi
# ==============================================================================

if __name__ == "__main__":

    # Pastikan OpenCV tersedia untuk benchmarking
    try:
        import cv2
    except ImportError:
        print("ERROR: OpenCV (cv2) not installed. Cannot run benchmark.")
        exit()

    # Inisialisasi CV2 untuk menghindari error
    try:
        # Coba inisialisasi modul time/CV2
        pass
    except Exception:
        pass

    # Train model
    model, results, best_model_path = train_model()

    # Benchmark inference
    benchmark_inference(best_model_path)

    # Export optimized model
    export_optimized_model(best_model_path)

    # Analyze performance
    analyze_model_performance(best_model_path, CONFIG['data'])

    # Finish W&B run
    if wandb.run:
        wandb.finish()

    print("\n" + "=" * 50)
    print("✅ Training Complete!")
    print("=" * 50)
    print(f"Best model: {best_model_path}")
    print("\nNext steps:")
    print("1. Check W&B dashboard for detailed metrics")
    print("2. Test inference on sample images")
    print("3. Deploy to edge device for real-world testing")

🚀 Bottle Cap Detection Training (YOLOv8n)
Dataset Size: ~100. Augmentation is CRUCIAL.
Device: cpu
Base Model: yolov8n.pt (Pre-trained)
Fine-Tuning LR: 0.0001
Patience (Early Stop): 50


Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Bottle-Cap-Detection-12/data.yaml, degrees=15.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=yolov8n_cap_20251125_140200, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=50, per

final_mAP50,▁
final_mAP50-95,▁
final_mAP50,0.995
final_mAP50-95,0.86363



✅ Training Complete!
Best model: runs/train_bottle_cap/yolov8n_cap_20251125_140200/weights/best.pt

Next steps:
1. Check W&B dashboard for detailed metrics
2. Test inference on sample images
3. Deploy to edge device for real-world testing


# **5. Inference with Pytorch Model**

In [13]:
import cv2
import time
import numpy as np
from ultralytics import YOLO


# ================================
# USER CONFIG — EDIT DI SINI
# ================================
PATH_IMAGE = "/content/tes11.jpg"        # path ke gambar input
PATH_MODEL = "/content/runs/train_bottle_cap/yolov8n_cap_20251125_140200/weights/best.pt"           # path ke YOLO model
IMG_SIZE   = 320
CONF_THRESH = 0.5
OUTPUT_PATH = "output.jpg"
RUN_BENCHMARK = False
BENCHMARK_RUNS = 100
# ================================


class BottleCapDetector:
    """Bottle cap detector with color classification"""

    def __init__(self, model_path: str, imgsz: int = 320, conf: float = 0.5):
        self.model = YOLO(model_path)
        self.imgsz = imgsz
        self.conf = conf
        self.class_names = ['dark_blue', 'light_blue', 'others']
        self.colors = {
            0: (139, 0, 0),      # dark_blue BGR
            1: (255, 200, 100),  # light_blue BGR
            2: (128, 128, 128)   # others
        }

    def detect(self, image_path: str, visualize: bool = True):
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Cannot load image: {image_path}")

        start_time = time.time()
        results = self.model(img, imgsz=self.imgsz, conf=self.conf, verbose=False)
        inference_time = (time.time() - start_time) * 1000

        detections = []
        for r in results:
            for box in r.boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                xyxy = box.xyxy[0].cpu().numpy()

                detections.append({
                    'class': self.class_names[cls],
                    'confidence': conf,
                    'bbox': xyxy.tolist()
                })

        output_img = None
        if visualize:
            output_img = self._visualize(img.copy(), detections)

        return {
            'detections': detections,
            'inference_time_ms': inference_time,
            'image': output_img
        }

    def _visualize(self, img, detections):
        for det in detections:
            x1, y1, x2, y2 = map(int, det['bbox'])
            class_name = det['class']
            conf = det['confidence']

            color = self.colors[self.class_names.index(class_name)]
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

            label = f"{class_name}: {conf:.2f}"
            label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)

            cv2.rectangle(img,
                          (x1, y1 - label_size[1] - 10),
                          (x1 + label_size[0], y1),
                          color, -1)

            cv2.putText(img, label, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (255, 255, 255), 2)

        return img

    def benchmark(self, image_path: str, num_runs: int = 100):
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Cannot load image: {image_path}")

        for _ in range(10):
            _ = self.model(img, imgsz=self.imgsz, verbose=False)

        times = []
        for _ in range(num_runs):
            start = time.time()
            _ = self.model(img, imgsz=self.imgsz, verbose=False)
            times.append((time.time() - start) * 1000)

        return {
            'avg_ms': np.mean(times),
            'min_ms': np.min(times),
            'max_ms': np.max(times),
            'std_ms': np.std(times),
            'fps': 1000 / np.mean(times)
        }


def main():
    print("=" * 60)
    print(" Bottle Cap Detector — Auto Run Mode")
    print("=" * 60)
    print(f"Model: {PATH_MODEL}")
    print(f"Image: {PATH_IMAGE}")

    detector = BottleCapDetector(PATH_MODEL, IMG_SIZE, CONF_THRESH)

    print("\n Running detection...")
    result = detector.detect(PATH_IMAGE, visualize=True)

    print("\n Results:")
    print(f"Inference time: {result['inference_time_ms']:.2f} ms")
    print(f"Detections: {len(result['detections'])}")

    if result["detections"]:
        print("\n Detected objects:")
        for i, det in enumerate(result["detections"], 1):
            print(f"  {i}. {det['class']} — {det['confidence']:.3f}")
    else:
        print(" No objects detected.")

    if result["image"] is not None:
        cv2.imwrite(OUTPUT_PATH, result["image"])
        print(f"\n Output saved to: {OUTPUT_PATH}")

    if RUN_BENCHMARK:
        print("\n Running benchmark...")
        stats = detector.benchmark(PATH_IMAGE, BENCHMARK_RUNS)

        print("\n Benchmark Results:")
        for k, v in stats.items():
            print(f"  {k}: {v:.2f}")

    print("\n Done!")
    print("=" * 60)


if __name__ == "__main__":
    main()


 Bottle Cap Detector — Auto Run Mode
Model: /content/runs/train_bottle_cap/yolov8n_cap_20251125_140200/weights/best.pt
Image: /content/tes11.jpg

 Running detection...

 Results:
Inference time: 123.59 ms
Detections: 10

 Detected objects:
  1. light_blue — 0.966
  2. light_blue — 0.952
  3. light_blue — 0.934
  4. light_blue — 0.932
  5. dark_blue — 0.926
  6. light_blue — 0.924
  7. dark_blue — 0.918
  8. dark_blue — 0.915
  9. dark_blue — 0.885
  10. dark_blue — 0.858

 Output saved to: output.jpg

 Done!


# **6. Inference with ONNX Model**

In [14]:

import cv2
import time
import numpy as np
import onnxruntime as ort


# =====================================================
# USER CONFIG — EDIT DI SINI
# =====================================================
PATH_IMAGE = "/content/tes11.jpg"            # Path ke gambar input
PATH_MODEL = "/content/runs/train_bottle_cap/yolov8n_cap_20251125_140200/weights/best.onnx"            # Path ke ONNX model
IMG_SIZE = 320                       # Image size
CONF_THRESH = 0.5                    # Confidence threshold
OUTPUT_PATH = "output_fast.jpg"      # Output result
RUN_BENCHMARK = False                # True = jalankan benchmark
BENCHMARK_RUNS = 100                 # Jumlah iterasi benchmark
# =====================================================


class FastBottleCapDetector:
    """Optimized bottle cap detector using ONNX Runtime"""

    def __init__(self, model_path: str, imgsz: int = 320, conf: float = 0.5):
        self.imgsz = imgsz
        self.conf = conf
        self.class_names = ['dark_blue', 'light_blue', 'others']
        self.colors = {
            0: (139, 0, 0),
            1: (255, 200, 100),
            2: (128, 128, 128)
        }

        # Setup ONNX Runtime
        sess_options = ort.SessionOptions()
        sess_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
        sess_options.intra_op_num_threads = 4
        sess_options.inter_op_num_threads = 4

        providers = ['CPUExecutionProvider']

        if 'CUDAExecutionProvider' in ort.get_available_providers():
            providers.insert(0, 'CUDAExecutionProvider')
            print("Using CUDA acceleration")
        else:
            print("Using CPU")

        self.session = ort.InferenceSession(
            model_path,
            sess_options=sess_options,
            providers=providers
        )

        self.input_name = self.session.get_inputs()[0].name
        self.output_names = [out.name for out in self.session.get_outputs()]

        print(f"Model loaded: {model_path}")
        print(f"Input shape: {self.session.get_inputs()[0].shape}")
        print(f"Output shape: {self.session.get_outputs()[0].shape}")

    def preprocess(self, img):
        img_h, img_w = img.shape[:2]
        scale = min(self.imgsz / img_w, self.imgsz / img_h)
        new_w, new_h = int(img_w * scale), int(img_h * scale)

        img_resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

        img_padded = np.full((self.imgsz, self.imgsz, 3), 114, dtype=np.uint8)

        pad_w = (self.imgsz - new_w) // 2
        pad_h = (self.imgsz - new_h) // 2

        img_padded[pad_h:pad_h+new_h, pad_w:pad_w+new_w] = img_resized
        img_rgb = cv2.cvtColor(img_padded, cv2.COLOR_BGR2RGB)
        img_normalized = img_rgb.astype(np.float32) / 255.0
        img_transposed = np.transpose(img_normalized, (2, 0, 1))
        img_batch = np.expand_dims(img_transposed, 0).astype(np.float32)

        return img_batch, scale, (pad_w, pad_h)

    def postprocess(self, outputs, orig_shape, scale, padding):
        output = outputs[0]  # (1, 7, 2100)
        predictions = output[0].T  # (2100, 7)

        boxes_xywh = predictions[:, :4]
        scores = predictions[:, 4:]

        class_ids = np.argmax(scores, axis=1)
        confidences = np.max(scores, axis=1)

        mask = confidences > self.conf
        boxes_xywh = boxes_xywh[mask]
        class_ids = class_ids[mask]
        confidences = confidences[mask]

        if len(boxes_xywh) == 0:
            return []

        boxes_xyxy = np.zeros_like(boxes_xywh)
        boxes_xyxy[:, 0] = boxes_xywh[:, 0] - boxes_xywh[:, 2] / 2
        boxes_xyxy[:, 1] = boxes_xywh[:, 1] - boxes_xywh[:, 3] / 2
        boxes_xyxy[:, 2] = boxes_xywh[:, 0] + boxes_xywh[:, 2] / 2
        boxes_xyxy[:, 3] = boxes_xywh[:, 1] + boxes_xywh[:, 3] / 2

        pad_w, pad_h = padding
        boxes_xyxy[:, [0, 2]] -= pad_w
        boxes_xyxy[:, [1, 3]] -= pad_h

        boxes_xyxy /= scale

        boxes_xyxy[:, [0, 2]] = np.clip(boxes_xyxy[:, [0, 2]], 0, orig_shape[1])
        boxes_xyxy[:, [1, 3]] = np.clip(boxes_xyxy[:, [1, 3]], 0, orig_shape[0])

        indices = self.nms(boxes_xyxy, confidences)

        detections = []
        for idx in indices:
            detections.append({
                'class': self.class_names[class_ids[idx]],
                'confidence': float(confidences[idx]),
                'bbox': boxes_xyxy[idx].tolist()
            })

        return detections

    def nms(self, boxes, scores, iou_threshold=0.45):
        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]

        areas = (x2 - x1) * (y2 - y1)
        order = scores.argsort()[::-1]

        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)

            xx1 = np.maximum(x1[i], x1[order[1:]])
            yy1 = np.maximum(y1[i], y1[order[1:]])
            xx2 = np.minimum(x2[i], x2[order[1:]])
            yy2 = np.minimum(y2[i], y2[order[1:]])

            w = np.maximum(0.0, xx2 - xx1)
            h = np.maximum(0.0, yy2 - yy1)
            inter = w * h

            iou = inter / (areas[i] + areas[order[1:]] - inter)
            order = order[np.where(iou <= iou_threshold)[0] + 1]

        return keep

    def detect(self, image_path: str, visualize: bool = True):
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Cannot load image: {image_path}")

        orig_shape = img.shape[:2]
        input_tensor, scale, padding = self.preprocess(img)

        start = time.perf_counter()
        outputs = self.session.run(self.output_names, {self.input_name: input_tensor})
        inference_time = (time.perf_counter() - start) * 1000

        detections = self.postprocess(outputs, orig_shape, scale, padding)

        output_img = img.copy() if visualize else None

        if visualize:
            for det in detections:
                x1, y1, x2, y2 = map(int, det['bbox'])

                color = self.colors[self.class_names.index(det['class'])]
                cv2.rectangle(output_img, (x1, y1), (x2, y2), color, 2)

                label = f"{det['class']}: {det['confidence']:.2f}"
                cv2.putText(output_img, label, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (255, 255, 255), 2)

        return {
            'detections': detections,
            'inference_time_ms': inference_time,
            'image': output_img
        }

    def benchmark(self, image_path: str, num_runs: int = 100):
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Cannot load image: {image_path}")

        input_tensor, _, _ = self.preprocess(img)

        for _ in range(20):
            _ = self.session.run(self.output_names, {self.input_name: input_tensor})

        times = []
        for _ in range(num_runs):
            start = time.perf_counter()
            _ = self.session.run(self.output_names, {self.input_name: input_tensor})
            times.append((time.perf_counter() - start) * 1000)

        times = np.array(times)

        return {
            'avg_ms': np.mean(times),
            'min_ms': np.min(times),
            'max_ms': np.max(times),
            'std_ms': np.std(times),
            'fps': 1000 / np.mean(times)
        }


def main():
    print("=" * 70)
    print("FAST BOTTLE CAP DETECTOR — AUTO RUN MODE")
    print("=" * 70)
    print(f"Model: {PATH_MODEL}")
    print(f"Image: {PATH_IMAGE}")

    detector = FastBottleCapDetector(PATH_MODEL, IMG_SIZE, CONF_THRESH)

    print("\n Running detection...")
    result = detector.detect(PATH_IMAGE, visualize=True)

    print("\n Results:")
    print(f"  Inference time: {result['inference_time_ms']:.2f} ms")
    print(f"  Detections: {len(result['detections'])}")

    for i, det in enumerate(result['detections'], 1):
        print(f"  {i}. {det['class']} — {det['confidence']:.3f}")

    if result["image"] is not None:
        cv2.imwrite(OUTPUT_PATH, result["image"])
        print(f"\n Output saved to: {OUTPUT_PATH}")

    if RUN_BENCHMARK:
        print("\n Running benchmark...")
        stats = detector.benchmark(PATH_IMAGE, BENCHMARK_RUNS)

        print("\n Benchmark Results:")
        for k, v in stats.items():
            print(f"  {k}: {v:.2f}")

    print("\n DONE!")
    print("=" * 70)


if __name__ == "__main__":
    main()


FAST BOTTLE CAP DETECTOR — AUTO RUN MODE
Model: /content/runs/train_bottle_cap/yolov8n_cap_20251125_140200/weights/best.onnx
Image: /content/tes11.jpg
Using CPU
Model loaded: /content/runs/train_bottle_cap/yolov8n_cap_20251125_140200/weights/best.onnx
Input shape: [1, 3, 320, 320]
Output shape: [1, 7, 2100]

 Running detection...

 Results:
  Inference time: 144.65 ms
  Detections: 10
  1. light_blue — 0.966
  2. light_blue — 0.952
  3. light_blue — 0.934
  4. light_blue — 0.932
  5. dark_blue — 0.926
  6. light_blue — 0.924
  7. dark_blue — 0.918
  8. dark_blue — 0.915
  9. dark_blue — 0.885
  10. dark_blue — 0.858

 Output saved to: output_fast.jpg

 DONE!
